In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
df= pd.read_csv('./midterm1/dataset/Audiobooks_data.csv')

In [3]:
df.head()

,873,2160,2160.1,10.13,10.13.1,0,8.91,0.1,0.2,0.3,0.4,1
0,611,1404.0,2808,6.66,13.33,1,6.5,0.0,0.0,0,182,1
1,705,324.0,324,10.13,10.13,1,9.0,0.0,0.0,1,334,1
2,391,1620.0,1620,15.31,15.31,0,9.0,0.0,0.0,0,183,1
3,819,432.0,1296,7.11,21.33,1,9.0,0.0,0.0,0,0,1
4,138,2160.0,2160,10.13,10.13,1,9.0,0.0,0.0,0,5,1


In [4]:
X= df.iloc[:, 1:-1].copy()

In [5]:
X.shape

(14083, 10)

In [6]:
y= df.iloc[:, -1:]

In [7]:
X.head()

,2160,2160.1,10.13,10.13.1,0,8.91,0.1,0.2,0.3,0.4
0,1404.0,2808,6.66,13.33,1,6.5,0.0,0.0,0,182
1,324.0,324,10.13,10.13,1,9.0,0.0,0.0,1,334
2,1620.0,1620,15.31,15.31,0,9.0,0.0,0.0,0,183
3,432.0,1296,7.11,21.33,1,9.0,0.0,0.0,0,0
4,2160.0,2160,10.13,10.13,1,9.0,0.0,0.0,0,5


In [8]:
y.head()

,1
0,1
1,1
2,1
3,1
4,1


In [9]:
y_act= y.squeeze()

In [10]:
y_act.head()

0    1
1    1
2    1
3    1
4    1
Name: 1, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test1, y_train, y_test1 = train_test_split(X, y, test_size=0.2, random_state=0)

In [13]:
X_val, X_test, y_val, y_test= train_test_split(X_test1,y_test1,test_size=0.5,random_state=0)

In [14]:
print(X_train.head())
print(y_train.head())

         2160  2160.1  10.13  10.13.1  0  8.91   0.1    0.2  0.3  0.4
10156  2160.0    2160   8.00     8.00  0  8.91  0.00    0.0    0    0
10906  1620.0    1620   5.33     5.33  0  8.91  0.04    0.0    0  239
13371  2160.0    2160   5.33     5.33  0  8.91  0.00    0.0    0    0
4064   1188.0    1188   6.93     6.93  0  8.91  0.01  475.2    0   96
13293  2160.0    2160   5.74     5.74  0  8.91  0.03    0.0    0  166
       1
10156  0
10906  0
13371  0
4064   0
13293  0


In [15]:
print(X_test.head())
print(y_test.head())

         2160  2160.1  10.13  10.13.1  0  8.91   0.1    0.2  0.3  0.4
11475  2160.0    2160   5.33     5.33  0  8.91  0.01   0.00    0   22
11318  1620.0    1620   4.10     4.10  0  8.91  0.00   0.00    0    0
10135   216.0     216   8.00     8.00  0  8.91  0.00   0.00    0    0
654    1188.0    1188   8.00     8.00  0  8.91  0.00  47.52    0    0
6711   1620.0    1620  10.13    10.13  0  8.91  0.09   0.00    0    0
       1
11475  0
11318  0
10135  0
654    1
6711   0


In [16]:
train_data= tf.data.Dataset.from_tensor_slices((X_train.values,y_train.values))
validation_data= tf.data.Dataset.from_tensor_slices((X_val.values,y_val.values))
test_data= tf.data.Dataset.from_tensor_slices((X_test.values,y_test.values))

In [17]:
hidden_layer_size=8
LAYERS=[
    
    # Input layer
    tf.keras.layers.Dense(10, activation='relu'),
    
    # Hidden Layers
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    
    # Output Layer
    tf.keras.layers.Dense(2, activation='softmax')
]

In [18]:
def return_model(model_layers):
    model = tf.keras.Sequential(model_layers)
    return model

In [19]:
def get_score(model,test_data):
    return model.evaluate(test_data, steps=10, verbose=0)

## Hyperparameters: learning rate, epochs

In [20]:
import gc
def fit_with(model_layers,train_data,val_data,lr,epochs):
        model=return_model(model_layers)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

        # Train the model with the train dataset.
        model.fit(x=train_data, validation_data=val_data, epochs=int(epochs), verbose=2)

        # Evaluate the model with the eval dataset.
        score = get_score(model,val_data)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        # Return the accuracy.
        del model
        for i in range(10):
            gc.collect()
        return score[1]
    
from functools import partial

verbose = 1
fit_with_partial = partial(fit_with, LAYERS, train_data, validation_data)

In [22]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'lr': (1e-6, 1e-3),
          'epochs': (1, 20)}

optimizer = BayesianOptimization(
    f=fit_with_partial,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

optimizer.maximize(init_points=5, n_iter=10,)


for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.max)

|   iter    |  target   |  epochs   |    lr     |
-------------------------------------------------
Train for 11266 steps, validate for 1408 steps
Epoch 1/8


KeyboardInterrupt: 

In [21]:
best_lr = optimizer.max['params']['lr']
best_epochs = optimizer.max['params']['epochs']
print(optimizer.max)

NameError: name 'optimizer' is not defined

In [ ]:
# Find Dimensions

batch_size = 20
epochs = int(best_epochs)

model = return_model(LAYERS)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_lr), 
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.fit(x=train_data,
         epochs=epochs,
         validation_data=test_data,
         validation_steps=10,
         verbose=2
         )